 # Transaction generation

## 1. Import required libraries

In [1]:
if(!is.element("devtools", installed.packages()[,1])){
  install.packages("devtools")
}

if(!is.element("Rcpp", installed.packages()[,1])){
  devtools::install_github("RcppCore/Rcpp")
}

if(!is.element("DBI", installed.packages()[,1])){
  devtools::install_github("rstats-db/DBI")
}

if(!is.element("RPostgres", installed.packages()[,1])){
  devtools::install_github("rstats-db/RPostgres")
}

if(!is.element("uuid", installed.packages()[,1])){
  install.packages("uuid")
}


library(RPostgres)
library(DBI)
library(uuid)

## 2. Define constants

In [2]:
banks <- c("bandit-radinou", "bandit-picsou", "bandit-donsaluste", "bandit-profit")

## 3. Generate transactions

In [3]:
generate_birth_date <- function(n) {
  # Générer des années de naissance aléatoires entre 1970 et 2005
  annees <- sample(1930:2005, n, replace = TRUE)

  # Générer des mois de naissance aléatoires entre janvier et décembre
  mois <- sample(1:12, n, replace = TRUE)

  # Générer des jours de naissance aléatoires entre 1 et 28
  jours <- sample(1:28, n, replace = TRUE)

  # Générer des heures de naissance aléatoires entre 0 et 23
  heures <- sample(0:23, n, replace = TRUE)

  # Générer des minutes de naissance aléatoires entre 0 et 59
  minutes <- sample(0:59, n, replace = TRUE)

  # Générer des secondes de naissance aléatoires entre 0 et 59
  secondes <- sample(0:59, n, replace = TRUE)

  # Combiner les années, mois, jours, heures, minutes et secondes en un vecteur de dates-temps
  ClientBirthDate <- as.POSIXct(paste(annees, mois, jours, heures, minutes, secondes, sep = "-"), format = "%Y-%m-%d-%H-%M-%S")
  return (as.data.frame(ClientBirthDate))
}

In [4]:
generate_transaction_date <- function(n) {
  start_date <- as.POSIXct("2023-01-01-0-0-0", format="%Y-%m-%d-%H-%M-%S")
  end_date <- Sys.time()
  TransactionDate <- as.POSIXct(start_date + runif(n, 0, difftime(end_date, start_date, units = "secs")), origin="1970-01-01")
  return (as.data.frame(TransactionDate))
}

In [5]:
temp_data <- read.csv("../workingsets/temp_age_gender_status_salary_bank_activity_amount_challenge.csv", header=TRUE)

Id <- unlist(lapply(seq_len(nrow(temp_data)), function(i) { UUIDgenerate() }))
DebitBank <- unlist(lapply(seq_len(nrow(temp_data)), function(i) { sample(banks,1,replace = TRUE, prob = c(0.1, 0.2, 0.3, 0.4)) }))
CreditBank <- temp_data$bank
ClientId <- unlist(lapply(seq_len(nrow(temp_data)), function(i) { UUIDgenerate() }))
ClientGender <- temp_data$gender
ClientBirthDate <- generate_birth_date(nrow(temp_data))
ClientAge <- temp_data$age
ClientMaritalStatus <- temp_data$status
ClientMonthlySalary <- temp_data$salary
TransactionDate <- generate_transaction_date(nrow(temp_data))
MerchantActivity <- temp_data$activity
AuthenticationMethod <- temp_data$challenge
TransferredAmount <- temp_data$amount

## 3. Export transactions

In [6]:
transactions <- data.frame(Id,DebitBank,CreditBank,ClientId,ClientGender,ClientBirthDate = ClientBirthDate,ClientAge,ClientMaritalStatus,ClientMonthlySalary,TransactionDate = TransactionDate,MerchantActivity,AuthenticationMethod,TransferredAmount)
write.csv(transactions, file = "../workingsets/transactions.csv")

con <- dbConnect(RPostgres::Postgres(), host='localhost', port='5433', dbname='bandit-nbs', user='bandit', password="orF9YuPWVajej5tC6cfiro94BoxrzsoE")
dbWriteTable(con, "Transaction", transactions, append = TRUE, row.names = FALSE)